<a href="https://colab.research.google.com/github/Melo987/-v2/blob/main/classify-leaves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U d2l

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install matplotlib==3.0.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install matplotlib_inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
!pip install ttach

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
!pip install git+https://github.com/ildoonet/cutmix 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ildoonet/cutmix to /tmp/pip-req-build-j9afwfew
  Running command git clone -q https://github.com/ildoonet/cutmix /tmp/pip-req-build-j9afwfew
  Created wheel for cutmix: filename=cutmix-0.1-py3-none-any.whl size=3599 sha256=40ac0a915f4f7b4f1fcc57977843064538bd03995c867256c0786583d67b5404
  Stored in directory: /tmp/pip-ephem-wheel-cache-7d4joqcy/wheels/a9/81/a7/d3822499b14d97b1e2ef7e7538b70f15355607cfc7526f7cd5
Successfully built cutmix


In [28]:
# 安装ResNeSt模型包
!pip install resnest --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 5.0 MB/s 
     |████████████████████████████████| 154 kB 42.3 MB/s 
     |████████████████████████████████| 50 kB 3.4 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=3866c6e8aa42fe8e8bff50445bfc53961168c2fec0e8d4c555ca75d7234a9ac2
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31549 sha256=10544d2f2ed3196fc45ef124fc79b5ad1373a2d3396eb456e4d0eb87e15ad7ec
  Stored in directory: /root/.cache/pip/wheels/aa/cc/ed/ca4e88beef656b01c84b9185196513ef2faf74a5a379b043a7
Successfully built fvcore iopath


In [29]:
# 导入各种包
import torch
import torch.nn as nn
from torch.nn import functional as F
import ttach as tta
from resnest.torch import resnest50

from cutmix.cutmix import CutMix
from cutmix.utils import CutMixCrossEntropyLoss

import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import KFold
from PIL import Image
import os
import matplotlib.pyplot as plt
import torchvision.models as models
# This is for the progress bar.
from tqdm import tqdm

In [6]:
# 如果你没有安装pandas，请取消下一行的注释
# !pip install pandas

%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [20]:
train_iter = pd.read_csv('/content/classify-leaves/train.csv')
test_iter = pd.read_csv('/content/classify-leaves/test.csv')

In [8]:
print(train_iter.shape)
print(test_iter.shape)

(18353, 2)
(8800, 1)


In [9]:
print(train_iter.iloc[:])

                  image                    label
0          images/0.jpg         maclura_pomifera
1          images/1.jpg         maclura_pomifera
2          images/2.jpg         maclura_pomifera
3          images/3.jpg         maclura_pomifera
4          images/4.jpg         maclura_pomifera
...                 ...                      ...
18348  images/18348.jpg          aesculus_glabra
18349  images/18349.jpg  liquidambar_styraciflua
18350  images/18350.jpg            cedrus_libani
18351  images/18351.jpg      prunus_pensylvanica
18352  images/18352.jpg          quercus_montana

[18353 rows x 2 columns]


In [10]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import matplotlib.pyplot as plt
import torchvision.models as models
# This is for the progress bar.
from tqdm import tqdm
import seaborn as sns

In [21]:
# 把label文件排个序
# set() 函数创建一个无序不重复元素集
# list() 创建列表
leaves_labels = sorted(list(set(train_iter['label'])))
n_classes = len(leaves_labels)
print(n_classes)
leaves_labels[:10]

176


['abies_concolor',
 'abies_nordmanniana',
 'acer_campestre',
 'acer_ginnala',
 'acer_griseum',
 'acer_negundo',
 'acer_palmatum',
 'acer_pensylvanicum',
 'acer_platanoides',
 'acer_pseudoplatanus']

In [22]:
class_to_num = dict(zip(leaves_labels, range(n_classes)))
class_to_num

{'abies_concolor': 0,
 'abies_nordmanniana': 1,
 'acer_campestre': 2,
 'acer_ginnala': 3,
 'acer_griseum': 4,
 'acer_negundo': 5,
 'acer_palmatum': 6,
 'acer_pensylvanicum': 7,
 'acer_platanoides': 8,
 'acer_pseudoplatanus': 9,
 'acer_rubrum': 10,
 'acer_saccharinum': 11,
 'acer_saccharum': 12,
 'aesculus_flava': 13,
 'aesculus_glabra': 14,
 'aesculus_hippocastamon': 15,
 'aesculus_pavi': 16,
 'ailanthus_altissima': 17,
 'albizia_julibrissin': 18,
 'amelanchier_arborea': 19,
 'amelanchier_canadensis': 20,
 'amelanchier_laevis': 21,
 'asimina_triloba': 22,
 'betula_alleghaniensis': 23,
 'betula_jacqemontii': 24,
 'betula_lenta': 25,
 'betula_nigra': 26,
 'betula_populifolia': 27,
 'broussonettia_papyrifera': 28,
 'carpinus_betulus': 29,
 'carpinus_caroliniana': 30,
 'carya_cordiformis': 31,
 'carya_glabra': 32,
 'carya_ovata': 33,
 'carya_tomentosa': 34,
 'castanea_dentata': 35,
 'catalpa_bignonioides': 36,
 'catalpa_speciosa': 37,
 'cedrus_atlantica': 38,
 'cedrus_deodara': 39,
 'cedru

In [23]:
# 再转换回来，方便最后预测的时候使用
num_to_class = {v : k for k, v in class_to_num.items()}
num_to_class

{0: 'abies_concolor',
 1: 'abies_nordmanniana',
 2: 'acer_campestre',
 3: 'acer_ginnala',
 4: 'acer_griseum',
 5: 'acer_negundo',
 6: 'acer_palmatum',
 7: 'acer_pensylvanicum',
 8: 'acer_platanoides',
 9: 'acer_pseudoplatanus',
 10: 'acer_rubrum',
 11: 'acer_saccharinum',
 12: 'acer_saccharum',
 13: 'aesculus_flava',
 14: 'aesculus_glabra',
 15: 'aesculus_hippocastamon',
 16: 'aesculus_pavi',
 17: 'ailanthus_altissima',
 18: 'albizia_julibrissin',
 19: 'amelanchier_arborea',
 20: 'amelanchier_canadensis',
 21: 'amelanchier_laevis',
 22: 'asimina_triloba',
 23: 'betula_alleghaniensis',
 24: 'betula_jacqemontii',
 25: 'betula_lenta',
 26: 'betula_nigra',
 27: 'betula_populifolia',
 28: 'broussonettia_papyrifera',
 29: 'carpinus_betulus',
 30: 'carpinus_caroliniana',
 31: 'carya_cordiformis',
 32: 'carya_glabra',
 33: 'carya_ovata',
 34: 'carya_tomentosa',
 35: 'castanea_dentata',
 36: 'catalpa_bignonioides',
 37: 'catalpa_speciosa',
 38: 'cedrus_atlantica',
 39: 'cedrus_deodara',
 40: 'c

In [15]:
# 继承pytorch的dataset，创建自己的
class TrainValidData(Dataset):
    def __init__(self, csv_path, file_path, resize_height=224, resize_width=224, transform=None):
        """
        Args:
            csv_path (string): csv 文件路径
            img_path (string): 图像文件所在路径

        """
        
        # 需要调整后的照片尺寸，我这里每张图片的大小尺寸不一致#
        self.resize_height = resize_height
        self.resize_width = resize_width

        self.file_path = file_path
        self.to_tensor = transforms.ToTensor() #将数据转换成tensor形式
        self.transform = transform

        # 读取 csv 文件
        # 利用pandas读取csv文件
        self.data_info = pd.read_csv(csv_path, header=None)  #header=None是去掉表头部分
        # 文件第一列包含图像文件名称
        self.image_arr = np.asarray(self.data_info.iloc[1:,0]) #self.data_info.iloc[1:,0]表示读取第一列，从第二行开始一直读取到最后一行
        # 第二列是图像的 label
        self.label_arr = np.asarray(self.data_info.iloc[1:,1])
        # 计算 length
        self.data_len = len(self.data_info.index) - 1

    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        single_image_name = self.image_arr[index]

        # 读取图像文件
        img_as_img = Image.open(self.file_path + single_image_name)
        
        #如果需要将RGB三通道的图片转换成灰度图片可参考下面两行
        # if img_as_img.mode != 'L':
        #     img_as_img = img_as_img.convert('L')
        
        #设置好需要转换的变量，还可以包括一系列的nomarlize等等操作
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        img_as_img = transform(img_as_img)

        # 得到图像的 label
        label = self.label_arr[index]
        number_label = class_to_num[label]

        return (img_as_img, number_label)  #返回每一个index对应的图片数据和对应的label

    def __len__(self):
        return self.data_len

In [17]:
# 继承pytorch的dataset，创建自己的
class TestData(Dataset):
    def __init__(self, csv_path, file_path, resize_height=224, resize_width=224, transform = None):
        """
        Args:
            csv_path (string): csv 文件路径
            img_path (string): 图像文件所在路径

        """
        
        # 需要调整后的照片尺寸，我这里每张图片的大小尺寸不一致#
        self.resize_height = resize_height
        self.resize_width = resize_width

        self.file_path = file_path
        self.transform = transform
        self.to_tensor = transforms.ToTensor() #将数据转换成tensor形式

        # 读取 csv 文件
        # 利用pandas读取csv文件
        self.data_info = pd.read_csv(csv_path, header=None)  #header=None是去掉表头部分
        # 文件第一列包含图像文件名称
        self.image_arr = np.asarray(self.data_info.iloc[1:,0]) #self.data_info.iloc[1:,0]表示读取第一列，从第二行开始一直读取到最后一行
        # 计算 length
        self.data_len = len(self.data_info.index) - 1
        
    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        single_image_name = self.image_arr[index]

        # 读取图像文件
        img_as_img = Image.open(self.file_path + single_image_name)
        
        #如果需要将RGB三通道的图片转换成灰度图片可参考下面两行
        # if img_as_img.mode != 'L':
        #     img_as_img = img_as_img.convert('L')
        
        #设置好需要转换的变量，还可以包括一系列的nomarlize等等操作
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        img_as_img = transform(img_as_img)


        return img_as_img

    def __len__(self):
        return self.data_len

In [18]:
train_transform = transforms.Compose([
    # 随机裁剪图像，所得图像为原始面积的0.08到1之间，高宽比在3/4和4/3之间。
    # 然后，缩放图像以创建224 x 224的新图像
    transforms.RandomResizedCrop(224, scale=(0.08, 1.0), ratio=(3.0 / 4.0, 4.0 / 3.0)),
    transforms.RandomHorizontalFlip(),
    # 随机更改亮度，对比度和饱和度
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    # 添加随机噪声
    transforms.ToTensor(),
    # 标准化图像的每个通道
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
val_test_transform = transforms.Compose([
    transforms.Resize(256),
    # 从图像中心裁切224x224大小的图片
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

In [24]:
train_val_path = '/content/classify-leaves/train.csv'
test_path = '/content/classify-leaves/test.csv'
# csv文件中已经images的路径了，因此这里只到上一级目录
img_path = '/content/classify-leaves'

train_val_dataset = TrainValidData(train_val_path, img_path)
test_dataset = TestData(test_path, img_path, transform = val_test_transform)
print(train_val_dataset.data_info)
print(test_dataset.data_info)

                      0                        1
0                 image                    label
1          images/0.jpg         maclura_pomifera
2          images/1.jpg         maclura_pomifera
3          images/2.jpg         maclura_pomifera
4          images/3.jpg         maclura_pomifera
...                 ...                      ...
18349  images/18348.jpg          aesculus_glabra
18350  images/18349.jpg  liquidambar_styraciflua
18351  images/18350.jpg            cedrus_libani
18352  images/18351.jpg      prunus_pensylvanica
18353  images/18352.jpg          quercus_montana

[18354 rows x 2 columns]
                     0
0                image
1     images/18353.jpg
2     images/18354.jpg
3     images/18355.jpg
4     images/18356.jpg
...                ...
8796  images/27148.jpg
8797  images/27149.jpg
8798  images/27150.jpg
8799  images/27151.jpg
8800  images/27152.jpg

[8801 rows x 1 columns]


In [49]:
# 是否要冻住模型的前面一些层
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        model = model
        for param in model.parameters():
            param.requires_grad = False

# ResNeSt模型
def resnest_model(num_classes, feature_extract = False):
    model_ft = resnest50(pretrained=True)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, num_classes))

    return model_ft

In [30]:
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device()
print(device)

cuda


In [31]:
!nvidia-smi

Sat Oct  8 04:09:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [45]:
# Configuration options
k_folds = 5
num_epochs = 30
learning_rate = 1e-4
weight_decay = 1e-3
train_loss_function = CutMixCrossEntropyLoss(True)
valid_loss_function = nn.CrossEntropyLoss()
# For fold results
results = {}

# Set fixed random number seed
torch.manual_seed(42)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

In [46]:
# Start print
print('--------------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids,valid_ids) in enumerate(kfold.split(train_val_dataset)):

  # Print
  print(f'FOLD {fold}')
  print('--------------------------------------')

  # Sample elements randomly from a given list of ids, no replacement.
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)

  # Define data loaders for training and testing data in this fold
  trainloader = torch.utils.data.DataLoader(
                      CutMix(TrainValidData(train_val_path, img_path, transform = train_transform), num_class=176, beta=1.0, prob=0.5, num_mix=2), 
                      batch_size=32, sampler=train_subsampler, num_workers=0)
  validloader = torch.utils.data.DataLoader(
                      TrainValidData(train_val_path, img_path, transform = val_test_transform),
                      batch_size=32, sampler=valid_subsampler, num_workers=0)
  
  # Initialize a model and put it on the device specified.
  model = resnest_model(176)
  model = model.to(device)
  model.device = device
  
  # Initialize optimizer
  optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate,weight_decay= weight_decay)
  scheduler = CosineAnnealingLR(optimizer,T_max=10)

  # Run the training loop for defined number of epochs
  for epoch in range(0,num_epochs):
    model.train()
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    # These are used to record information in training
    train_losses = []
    train_accs = []
    # Iterate the training set by batches
    for batch in tqdm(trainloader):
      # Move images and labels to GPU
      imgs, labels = batch
      imgs = imgs.to(device)
      labels = labels.to(device)
      # Forward the data
      logits = model(imgs)
      # Calculate loss
      loss = train_loss_function(logits,labels)
      # Clear gradients in previous step
      optimizer.zero_grad()
      # Compute gradients for parameters
      loss.backward()
      # Update the parameters with computed gradients
      optimizer.step()
      # Compute the accuracy for current batch.
      # acc = (logits.argmax(dim=-1) == labels).float().mean()
      # Record the loss and accuracy.
      train_losses.append(loss.item())
      # train_accs.append(acc)
    print("第%d个epoch的学习率：%f" % (epoch+1,optimizer.param_groups[0]['lr']))
    scheduler.step()
    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = np.sum(train_losses) / len(train_losses)
    # train_acc = np.sum(train_accs) / len(train_accs)
    # Print the information.
    # print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
    print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}")

  # Train process (all epochs) is complete
  print('Training process has finished. Saving trained model.')
  print('Starting validation')

  # Saving the model
  print('saving model with loss {:.3f}'.format(train_loss))
  save_path = f'./model-fold-{fold}.pth'
  torch.save(model.state_dict(),save_path)
  # Start Validation
  model.eval()
  valid_losses = []
  valid_accs = []
  with torch.no_grad():
    for batch in tqdm(validloader):
      imgs, labels = batch
      # No gradient in validation
      logits = model(imgs.to(device))
      loss = valid_loss_function(logits,labels.to(device))
      acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
      # Record loss and accuracy
      valid_losses.append(loss.item())        
      valid_accs.append(acc)
    # The average loss and accuracy
    valid_loss = np.sum(valid_losses)/len(valid_losses)
    valid_acc = np.sum(valid_accs)/len(valid_accs)
    print(f"[ Valid | {epoch + 1:03d}/{num_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
    print('Accuracy for fold %d: %d' % (fold, valid_acc))
    print('--------------------------------------')
    results[fold] = valid_acc
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
total_summation = 0.0
for key, value in results.items():
  print(f'Fold {key}: {value} ')
  total_summation += value
print(f'Average: {total_summation/len(results.items())} ')

--------------------------------------
FOLD 0
--------------------------------------
Starting epoch 1


  0%|          | 0/459 [00:00<?, ?it/s]


FileNotFoundError: ignored

In [47]:
testloader = torch.utils.data.DataLoader(
                      TestData(test_path, img_path, transform = val_test_transform),
                      batch_size=128, num_workers=0)

In [50]:
## predict
model = resnext_model(176)

# create model and load weights from checkpoint
model = model.to(device)
# load the all folds
for test_fold in range(k_folds):
  model_path = f'./model-fold-{test_fold}.pth'
  saveFileName = f'./submission-fold-{test_fold}.csv'
  model.load_state_dict(torch.load(model_path))

  # Make sure the model is in eval mode.
  # Some modules like Dropout or BatchNorm affect if the model is in training mode.
  model.eval()
  tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.five_crop_transform(200,200))

  # Initialize a list to store the predictions.
  predictions = []
  # Iterate the testing set by batches.
  for batch in tqdm(testloader):
      
      imgs = batch
      with torch.no_grad():
          logits = tta_model(imgs.to(device))
      
      # Take the class with greatest logit as prediction and record it.
      predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

  preds = []
  for i in predictions:
      preds.append(num_to_class[i])

  test_data = pd.read_csv(test_path)
  test_data['label'] = pd.Series(preds)
  submission = pd.concat([test_data['image'], test_data['label']], axis=1)
  submission.to_csv(saveFileName, index=False)
  print("ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!")

NameError: ignored

In [42]:
!unzip '/content/classify-leaves/images.zip' -d '/content/classify-leaves'

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/classify-leaves/images/5499.jpg  
  inflating: /content/classify-leaves/images/55.jpg  
  inflating: /content/classify-leaves/images/550.jpg  
  inflating: /content/classify-leaves/images/5500.jpg  
  inflating: /content/classify-leaves/images/5501.jpg  
  inflating: /content/classify-leaves/images/5502.jpg  
  inflating: /content/classify-leaves/images/5503.jpg  
  inflating: /content/classify-leaves/images/5504.jpg  
  inflating: /content/classify-leaves/images/5505.jpg  
  inflating: /content/classify-leaves/images/5506.jpg  
  inflating: /content/classify-leaves/images/5507.jpg  
  inflating: /content/classify-leaves/images/5508.jpg  
  inflating: /content/classify-leaves/images/5509.jpg  
  inflating: /content/classify-leaves/images/551.jpg  
  inflating: /content/classify-leaves/images/5510.jpg  
  inflating: /content/classify-leaves/images/5511.jpg  
  inflating: /content/classify-leaves/images/5512.jpg  
  inflating: /content/cla

In [38]:
!remove '/content/classify-leaves/images'

/bin/bash: remove: command not found


In [41]:
!rm -rf /content/classify-leaves/images